In [ ]:
import subprocess
import shlex
import re
import json
import pyinotify

import numpy as np
import multiprocessing as mp

from pathlib import Path
from scipy import stats
from tqdm import tqdm
from IPython.display import display, Markdown
from tempfile import TemporaryDirectory

In [ ]:
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")

In [ ]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

In [ ]:
class EventLogSink(pyinotify.ProcessEvent):
    def __init__(self, event_log):
        Path(event_log).mkdir(parents=True, exist_ok=True)
        
        self.log_size = 0
        
        self.wm = pyinotify.WatchManager()
        self.notifier = pyinotify.ThreadedNotifier(self.wm, self)
        
        self.notifier.start()
        
        self.wm.add_watch(event_log, pyinotify.IN_CREATE | pyinotify.IN_CLOSE_WRITE, rec=True)
        
    def stop(self):
        self.notifier.stop()
        
        return self.log_size
    
    def process_IN_CREATE(self, event):
        self.wm.add_watch(event.pathname, pyinotify.IN_CREATE | pyinotify.IN_CLOSE_WRITE)
    
    def process_IN_CLOSE_WRITE(self, event):
        path = Path(event.pathname)
        
        if path.exists():
            self.log_size += path.stat().st_size
            
            path.unlink()

In [ ]:
def simulate_reporting_dynamic(
    algorithm, speciation=0.001, seed=42, sample=1.0, radius=564, sigma=10.0, report_speciation=False, report_dispersal=False, log=False
):
    if report_dispersal:
        reporters = ["Execution()", "Biodiversity()", "Counter()"]
    elif report_speciation:
        reporters = ["Execution()", "Biodiversity()"]
    else:
        reporters = ["Execution()"]
        
    with TemporaryDirectory() as log_path:
        event_log = EventLogSink(log_path)
        
        config = "".join(f"""
        (
            speciation: {speciation},
            sample: {sample},
            seed: {seed},

            algorithm: {algorithm},

            scenario: AlmostInfinite(
                radius: {radius},
                sigma: {sigma},
            ),
            
            log: {f'"{log_path}"' if log else 'None'},

            reporters: [
                Plugin(
                    library: "{target_directory}/release/deps/libnecsim_plugins_common.so",
                    reporters: [{', '.join(reporters)}]
                )
            ],
        )
        """.split()).replace(",)", ")").replace(",]", "]")

        # Run the simulation
        result = subprocess.run(shlex.split(
            f"{target_directory}/release/rustcoalescence simulate '{config}'"
        ), check=True, capture_output=True, text=True)
        
        event_log_size = event_log.stop()
    
    match = EXECUTION_PATTERN.search(result.stdout)
    if match is None:
        print(result.stdout)
        print(result.stderr)
    initialisation = parse_time(match.group(1))
    execution = parse_time(match.group(2))
    cleanup = parse_time(match.group(3))
        
    return initialisation, execution, cleanup, event_log_size

In [ ]:
def simulate_reporting_compiled(speciation=0.001, seed=42, sample=1.0, radius=564, sigma=10.0, report_speciation=False, report_dispersal=False):
    if report_dispersal:
        reporting = "progress-speciation-dispersal"
    elif report_speciation:
        reporting = "progress-speciation"
    else:
        reporting = "progress-only"
        
    # Run the simulation
    result = subprocess.run(shlex.split(
        f"{target_directory}/release/analysis-performance-reporting --radius {radius} " +
        f"--sample {sample} --seed {seed} --sigma {sigma} --speciation {speciation} {reporting}"
    ), check=True, capture_output=True, text=True)
    
    match = EXECUTION_PATTERN.search(result.stdout)
    if match is None:
        print(result.stdout)
        print(result.stderr)
    initialisation = parse_time(match.group(1))
    execution = parse_time(match.group(2))
    cleanup = parse_time(match.group(3))
        
    return initialisation, execution, cleanup

In [ ]:
def batch_simulation_many_seeds(simulate, seeds, args=tuple(), kwargs=dict(), silent=False, processes=mp.cpu_count()):
    results = []

    with tqdm(total=len(seeds), disable=silent) as progress:
        def update_progress(result):
            results.append(result)

            progress.update()
        
        def update_error(err):
            print(err)

        with mp.Pool(processes) as pool:
            for seed in seeds:
                pool.apply_async(simulate, args, {**kwargs, "seed": seed}, update_progress, update_error)

            pool.close()
            pool.join()
    
    return results

In [ ]:
def format_bytes(b):
    if b < 1e3:
        return f"{int(b)}B"
    elif b < 1e6:
        return f"{np.round(b / 1e3, 2)}kB"
    elif b < 1e9:
        return f"{np.round(b / 1e6, 2)}MB"
    elif b < 1e12:
        return f"{np.round(b / 1e9, 2)}GB"
    else:
        return f"{np.round(b / 1e12, 2)}TB"

In [ ]:
subprocess.run(shlex.split(
    f"cargo build --manifest-path {target_directory}/../rustcoalescence/Cargo.toml --release "
    + "--features rustcoalescence-algorithms-monolithic,rustcoalescence-algorithms-independent,"
    + "rustcoalescence-algorithms-cuda"
), check=True, capture_output=True, text=True);

subprocess.run(shlex.split(
    "cargo build --release"
), check=True, capture_output=True, text=True);

In [ ]:
display(Markdown("# RAM Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("free"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# CPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("lscpu"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# GPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("nvidia-smi"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

In [ ]:
display(Markdown("# Reporting Performance:"))

seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=160)

for report_speciation, report_dispersal, title in [
    (False, False, "progress only"), (True, False, "progress + speciation"), (True, True, "progress + speciation + dispersal")
]:
    display(Markdown(f"## {title}:"))
    
    initialisations, executions, cleanups, event_log_sizes = zip(*batch_simulation_many_seeds(
        simulate_reporting_dynamic, seeds, args=("Classical()",), kwargs=dict(
            speciation=0.001, sample=1.0, radius=178, sigma=0.0, report_speciation=report_speciation, report_dispersal=report_dispersal
        ), silent=False
    ))
    display(Markdown(f"* Dynamic Plugins (CPU): {np.round(np.mean(executions), 2)}s ± {np.round(np.std(executions), 2)}s"))

    initialisations, executions, cleanups = zip(*batch_simulation_many_seeds(
        simulate_reporting_compiled, seeds, args=(), kwargs=dict(
            speciation=0.001, sample=1.0, radius=178, sigma=0.0, report_speciation=report_speciation, report_dispersal=report_dispersal
        ), silent=False
    ))
    display(Markdown(f"* Compiled Analysis (CPU): {np.round(np.mean(executions), 2)}s ± {np.round(np.std(executions), 2)}s"))

    initialisations, executions, cleanups, event_log_sizes = zip(*batch_simulation_many_seeds(
        simulate_reporting_dynamic, seeds, args=("Classical()",), kwargs=dict(
            speciation=0.001, sample=1.0, radius=178, sigma=0.0, report_speciation=report_speciation, report_dispersal=report_dispersal, log=True
        ), silent=False
    ))
    display(Markdown(f"* Dynamic Plugins + Log (CPU): {np.round(np.mean(executions), 2)}s ± {np.round(np.std(executions), 2)}s [{format_bytes(np.mean(event_log_sizes))} ± {format_bytes(np.std(event_log_sizes))}]"))
    
    initialisations, executions, cleanups, event_log_sizes = zip(*batch_simulation_many_seeds(
        simulate_reporting_dynamic, seeds, args=("CUDA(ptx_jit:true, parallelism_mode:Monolithic(event_slice:10000000))",), kwargs=dict(
            speciation=0.001, sample=1.0, radius=178, sigma=0.0, report_speciation=report_speciation, report_dispersal=report_dispersal
        ), silent=False, processes=1
    ))
    display(Markdown(f"* Dynamic Plugins (GPU): {np.round(np.mean(executions), 2)}s ± {np.round(np.std(executions), 2)}s"))